In [3]:
#
# light sequencer: action request sending to the avatar microservice 
#                 
#

#!/usr/bin/env python
import pika, sys, os, json

action_request = lambda manip, equip, ref: {          
                    "manipulation": manip,
                    "equipment": {
                        "type": equip,
                        "reference": ref
                        }
                     }

def callback(ch, method, properties, body):
    print(" [x] Received from avatar service : %s" % body.decode())
    if body.decode() in ["OK", "NOK"]:
        ch.stop_consuming()

def main(desc, ident, action_type):
    
    connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost'))
    channel = connection.channel()
    channel.exchange_declare(exchange='mars', exchange_type='topic')
    
    msg=json.dumps(action_request(desc,ident,action_type))
    channel.basic_publish(exchange='mars', 
                          routing_key="sequencer.hmi.request", 
                          body=msg)
    print(" [x] Sent to avatar service : %s" % msg)

    result = channel.queue_declare('', exclusive=True)
    queue_name = result.method.queue
    channel.queue_bind(exchange='mars', queue=queue_name, routing_key="hmi.sequencer.report")
    
    print(' [*] Waiting for report from HMI. To exit press CTRL+C') 
    channel.basic_consume(queue=queue_name, on_message_callback=callback, auto_ack=True)
    channel.start_consuming()
    connection.close()
    print(" [*] connection closed")

if __name__ == '__main__':
    try:
        main('LOAD','EFFECTOR','WEB_C_DRILLING')
    except KeyboardInterrupt:
        print('Interrupted')
        try:
            sys.exit(0)
        except SystemExit:
            os._exit(0)

 [x] Sent to avatar service : {"manipulation": "LOAD", "equipment": {"type": "EFFECTOR", "reference": "WEB_C_DRILLING"}}
 [*] Waiting for report from HMI. To exit press CTRL+C
 [x] Received from avatar service : OK
 [*] connection closed
